In [27]:
# SETUP

# Libraries
from google.cloud import bigquery
import pandas as pd
import numpy as np
# client location
client = bigquery.Client(location = "europe-north1")
print("Client creating using default project: {}".format(client.project))

Client creating using default project: gentle-pier-291608


In [2]:
model = "model1"
query = """
        SELECT *
        FROM df."""+model+"""
        LIMIT 2
    """
query_job = client.query(query, location= "europe-north1")  # API request - starts the query
m = query_job.to_dataframe()
cols = m.columns

In [3]:
cat_features = cols[0:5]
SMART = cols[5::]

In [9]:
# nulls 1st 5 col
q1 = pd.DataFrame()
for col in cat_features:
    query = """
        SELECT (COUNT(*) - COUNT("""+col+""")) AS N_null
        FROM df.model1
        """
    query_job = client.query(query, location= "europe-north1")  # API request - starts the query
    qi = query_job.to_dataframe()
    qi["Feature"] = col
    q1 = pd.concat([qi,q1], axis=0)
q1 = q1[["Feature", "N_null"]]
q1 = q1.reset_index(drop=True)
q1

,Feature,N_null
0,failure,0
1,capacity_bytes,0
2,model,0
3,serial_number,0
4,date,0


In [10]:
# min, max, avg and len of unique elements by columns
q2 = pd.DataFrame()
for col in SMART:
    query = """
        SELECT MIN("""+col+""") AS MIN,
            MAX("""+col+""") AS MAX,
            AVG("""+col+""") AS AVG,
            COUNT (DISTINCT """+col+""") AS len_unique,
            (COUNT(*) - COUNT("""+col+""")) AS N_null
        FROM df.model1
        """
    query_job = client.query(query, location= "europe-north1")  # API request - starts the query
    qi = query_job.to_dataframe()
    qi["col"] = col
    q2 = pd.concat([qi,q2], axis=0)

In [14]:
q2 = q2[['col','MIN', 'MAX', 'AVG', 'len_unique', 'N_null']]
q2.columns = ['Feature','Min', 'Max', 'Men', 'N unique values', 'N null']
q2 = q2.reset_index(drop=True)

In [19]:
q2.sort_values(by=['Feature'])

,Feature,Min,Max,Men,N unique values,N null
35,smart_10_normalized,100,100,1.000000e+02,1,251539
34,smart_10_raw,0,0,0.000000e+00,1,251539
33,smart_12_normalized,0,100,9.999959e+01,4,251539
32,smart_12_raw,0,1566375939,3.325437e+01,264,251539
31,smart_183_normalized,1,100,9.941290e+01,100,251539
30,smart_183_raw,0,64730,3.064159e+00,1047,251539
29,smart_184_normalized,24,100,9.998975e+01,41,251539
28,smart_184_raw,0,76,1.024818e-02,41,251539
27,smart_187_normalized,1,100,9.990130e+01,98,251539
26,smart_187_raw,0,65535,2.542049e-01,539,251539


In [25]:
# features to drop
drop = q2.loc[q2['N unique values'] == 1]["Feature"].values

In [32]:
selected = np.setdiff1d(cols,drop)

In [37]:
print(*selected, sep = ", ")

capacity_bytes, date, failure, model, serial_number, smart_12_normalized, smart_12_raw, smart_183_normalized, smart_183_raw, smart_184_normalized, smart_184_raw, smart_187_normalized, smart_187_raw, smart_188_normalized, smart_188_raw, smart_189_normalized, smart_189_raw, smart_190_normalized, smart_190_raw, smart_192_normalized, smart_192_raw, smart_193_normalized, smart_193_raw, smart_194_normalized, smart_194_raw, smart_197_normalized, smart_197_raw, smart_198_normalized, smart_198_raw, smart_199_normalized, smart_199_raw, smart_1_normalized, smart_1_raw, smart_240_normalized, smart_240_raw, smart_241_raw, smart_242_raw, smart_3_normalized, smart_4_normalized, smart_4_raw, smart_5_normalized, smart_5_raw, smart_7_normalized, smart_7_raw, smart_9_normalized, smart_9_raw


In [38]:
print(*cols, sep = ", ")

date, serial_number, model, capacity_bytes, failure, smart_1_normalized, smart_1_raw, smart_3_normalized, smart_3_raw, smart_4_normalized, smart_4_raw, smart_5_normalized, smart_5_raw, smart_7_normalized, smart_7_raw, smart_9_normalized, smart_9_raw, smart_10_normalized, smart_10_raw, smart_12_normalized, smart_12_raw, smart_183_normalized, smart_183_raw, smart_184_normalized, smart_184_raw, smart_187_normalized, smart_187_raw, smart_188_normalized, smart_188_raw, smart_189_normalized, smart_189_raw, smart_190_normalized, smart_190_raw, smart_191_normalized, smart_191_raw, smart_192_normalized, smart_192_raw, smart_193_normalized, smart_193_raw, smart_194_normalized, smart_194_raw, smart_197_normalized, smart_197_raw, smart_198_normalized, smart_198_raw, smart_199_normalized, smart_199_raw, smart_240_normalized, smart_240_raw, smart_241_normalized, smart_241_raw, smart_242_normalized, smart_242_raw


In [39]:
len(selected)

46